## 기울기

- 기울기란? 모든 변수의 편미분을 벡터로 정리한 것.
- 책내용 : 한 개 이상의 변수가 들어오는 경우를 대비해서 편미분 시켜주는 함수 소개
- 다음페이지는 파이썬 코드...!

$$f(x_0, x_1) = x_0^2 + x_1^2          \left(\frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1}\right)$$

In [18]:
# 기울기 구하는 함수 선언
import numpy as np

def function_2(x):
    return x[0]**2 + x[1]**2        

def numerical_gradient(f, x):
    h = 1e-4                        # 해석적 미분을 하기위한 수치
    grad = np.zeros_like(x)         # 빈 array 생성
    
    for idx in range(x.size):
        fxh1 = f(x)
        v = x[idx]
        x[idx] = v + h
        fxh1 = f(x)
        
        x[idx] = v - h
        fxh2 = f(x)
    
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = v
        
    return grad

In [19]:
# 기울기 결과 예시
import numpy as np
grad1 = numerical_gradient(function_2, np.array([3.0, 4.0]))
grad2 = numerical_gradient(function_2, np.array([0.0, 2.0]))
grad3 = numerical_gradient(function_2, np.array([3.0, 0.0]))

print(grad1)
print(grad2)
print(grad3)

[6. 8.]
[0. 4.]
[6. 0.]


- 그림 4-9 실습 (gradient_2d.py)
- 각 장소에서 기울기가 가리키는 방향은, 함수의 출력값을 가장 크게 줄이는 방향이다. 
- ->손실함수를 최소화 시키는 방향을 알수 있게 되었다.

## 경사법 (경사 하강법)

- 기울기를 이용해서 최적의 매개변수 (가중치와 편향) 를 찾아내는 방법 중 하나
- 최적의 매개변수란? 

- ->손실함수가 최솟값이 될때의 매개변수 값을 말함 (아래는 경사법의 수식)

$$ x_0 = x_0 - \eta{\frac{\partial f}{\partial x_0}}$$
$$ x_1 = x_1 - \eta{\frac{\partial f}{\partial x_1}}$$
$$\eta(에타) : 학습률$$

In [22]:
# 경사 하강법 함수
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    # f : 최적화 하려는 함수
    # init_x : 초깃값
    # lr : 학습률
    # step_num : 반복횟수
    
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

### WARNING_ (단점)
- 함수가 극솟값, 최솟값, 또 안장점(saddle point) 이 되는 장소에서는 기울기가 0 이다
  - 극솟값 : 극소적인 최솟값 (한정된 범위 내에서의 최솟값)
  - 안장점 : 어느방향에서는 극댓값, 어느방향에서는 극솟값
- 경사법의 단점은, 복잡한 함수에서 고원(plateau) 이라고하는 곳에 빠져서 학습이 제대로 진행이 안될 수 있음

### NOTE_ (참고)
- 경사법은 최솟값을 찾느냐 최댓값을 찾느냐에따라 이름이 달라진다
- 최솟값을 찾는 경사법 : 경사 하강법
- 최댓값을 찾는 경사법 : 경사 상승법
- 결론 : 본질적으로 중요하지 않다고함. 보통 경사 하강법으로 등장한다고합니다.

In [23]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [24]:
# 학습률이 너무 큰 예 : lr=10.0
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

# 학습률이 너무 작은 예: lr=1e-10
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

array([ 2.34235971e+12, -3.96091057e+12])


- 그림 4-10 gradient_method.py 실행

### NOTE_ (참고)
- 학습률과 같은 매개변수를 하이퍼파라미터라고 한다 (사람이 직접 설정해야하는 것임)


### 신경망에서의 기울기
$$
\begin{equation*}
\mathbf{W} =  \begin{pmatrix}
w_{11} & w_{21} & w_{31} \\
w_{12} & w_{22} & w_{32}
\end{pmatrix}
\end{equation*}
$$

$$
\begin{equation*}
\mathbf{\frac{\partial L}{\partial W}} =  \begin{pmatrix}
\frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial W_{21}} & \frac{\partial L}{\partial W_{31}} \\
\frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial W_{22}} & \frac{\partial L}{\partial W_{32}}
\end{pmatrix}
\end{equation*}
$$

In [41]:
import sys, os
sys.path.append(os.pardir)  
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient


class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 랜덤으로 w 생성

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [42]:
# 객체 선언
net = simpleNet()
print(net.W)

[[ 1.15875713 -2.43448087 -1.01921226]
 [ 2.15711139 -0.00924973  1.11667505]]


In [43]:
# 임의의 x varidables 선언 및 내적 진행
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[ 2.63665453 -1.46901328  0.39348018]


In [44]:
# 내적의 최댓값 인덱스
np.argmax(p)

0

In [46]:
# 정답 레이벌 t 선언 후 (원 핫 인코딩), 손실함수 계산하기
t = np.array([0,0,1])
net.loss(x, t)

2.3588208113426927

In [48]:
# 기울기 값 계산 
def f(W):
        return net.loss(x,t)

dW= numerical_gradient(f,net.W)
print(dW)

# 만약에, 경사 하강법을 적용한다면, 
# -> (직전의 가중치) - (기울기값*학습률) 

[[ 0.53447292  0.00880758 -0.54328051]
 [ 0.80170939  0.01321138 -0.81492076]]


### 학습 알고리즘 구현하기 

#### 0. 전제 : 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고함
#### 1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져옴. 이것을 미니배치라고 한다.
#### 2. 기울기 산출 : 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매배변수의 기울기를 구한다.
#### 3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.
#### 4. 반복 : 1~3 단계를 반복한다.

### 2층 신경망 구현하기

두 개의 레이어이므로 그 구성은 다음 그림과 같다. 

<img src=http://neuralnetworksanddeeplearning.com/images/tikz35.png>

#### 활성화 함수
활성화 함수는 보통 실수 전체를 정의역으로 시그모이드 함수, 램프 함수, 맥스 아웃 함수 등이 있다.
- 시그모이드 함수: 실수 전체를 정의역으로, (0, 1)을 치역으로 가진다.
$$ f(u) = \frac{1}{1 + e^{-u}} $$

- 쌍곡선 정접 함수: (-1, 1)의 치역을 갖는다.
$$ f(u) = tanh(u) $$

- 램프 함수 (ramp function, rectified linear function): u < 0인 부분을 0으로 바꾼 단순 함수이다. 단순하고 계산량이 적다. 학습이 빠르고 최종결과도 더 좋은 경우가 많아 가장 많이 사용되고 있다.
$$ f(u) = max(u, 0) $$

- 맥스아웃 함수: 각각의 총 입력을 유닛별로 따로 계산한 후, 그 중한다. 최대값을 유닛의 출력으로 한다.   
$$ f(u_j) = max (u_jk) (k=1,...,K) $$

- 항등 사상: 회귀 문제를 위한 신경망에서 사용한다.
$$ f(u) = u $$

#### ★**소프트맥스 함수: 클래스 분류를 위한 신경망에서 사용한다. 출력의 합이 항상 1이 된다. 모든 유닛의 총 입력으로부터 결정되는 점이 다른 활성화 함수와 다르다. 지수 함수에 따른 오버플로우를 방지하기 위해, 보통 입력값 중 최대값을 기준으로 정규화한다. **
$$ f(u) = \frac{\exp({u_k})}{\sum_{j=1}^{k} \exp({u_j})} $$

In [51]:
import sys, os
sys.path.append(os.pardir)  
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:
    # 초기설정
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): 
 
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    # 활성화 함수 적용 및 소프트맥스(항등함수) 적용
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # 손실함수 계산 부분
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 정확도 계산 부분
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # 기울기 계산 부분
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    # 입력데이터, 정답레이블 설정
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

In [55]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [56]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 1000번 돌린다. 
for i in range(iters_num):
    # 600000개의 훈련데이터에서 100개를 무작위로 선택한다. 
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask] # 무작위로 선택된 100개의 입력 데이터
    t_batch = t_train[batch_mask] # 그 결과값인 100개의 레이블 데이터
    
    # 미분에 의한 기울기 벡터를 구함: 속도가 매우 느리다.
    # grad = network.numerical_gradient(x_batch, t_batch)
    
    # 오차 역전파에 의해 기울기 벡터를 구함
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        # 각각의 가중치를 초기 랜덤값에서 음의 기울기 방향으로 가중치를 갱신한다. (즉, 하강한다.)
        # 이 때 학습률(learning rate)의 값으로 갱신량을 결정한다. 오버 피팅이 발생하지 않도록 적절한 값을 산정한다.
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

### 시험 데이터로 평가하기
### NOTE_ 
- 에폭(epoch) 은 하나의 단위이다.
- 훈련 데이터가 총 10,000개이고 100개를 하나의 미니배치로 학습할경우
- 1 에폭은 100회가 된다. 그러니깐, 모든 훈련 데이터를 '소진' 하는게 1 에폭이다. 
- 다음페이지는 실제 학습

In [58]:
import numpy as np
from dataset.mnist import load_mnist


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 60000개의 훈련 데이터에서 임의로 100개를 선택한다.
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc: {:0.3f}, test_acc: {:0.3f}".format(train_acc, test_acc))

train_acc: 0.102, test_acc: 0.101
train_acc: 0.792, test_acc: 0.796
train_acc: 0.878, test_acc: 0.882
train_acc: 0.898, test_acc: 0.902
train_acc: 0.910, test_acc: 0.912
train_acc: 0.916, test_acc: 0.918
train_acc: 0.921, test_acc: 0.922
train_acc: 0.926, test_acc: 0.926
train_acc: 0.929, test_acc: 0.930
train_acc: 0.932, test_acc: 0.934
train_acc: 0.935, test_acc: 0.937
train_acc: 0.938, test_acc: 0.938
train_acc: 0.941, test_acc: 0.939
train_acc: 0.943, test_acc: 0.942
train_acc: 0.945, test_acc: 0.945
train_acc: 0.946, test_acc: 0.946
train_acc: 0.948, test_acc: 0.948


### 정리
- 기계학습 사용 데이터는 훈련 데이터와 시험 데이터로 나눈다.
- 훈련 데이터로 학습한 모델의 범용 능력을 시험 데이터로 평가한다.
- 신경망 학습은 손실 함수를 지표로, 손실 함수 값이 작아지는 방향으로 매개변수를 갱신한다.
- 가중치 매개변수를 갱신할 때는 기울기를 이용하고, 기울어진 방향으로 가중치 값을 갱신하는 작업을 반복한다.
- 미분을 해서 가중치 매개변수의 기울기를 계산할 수 있다.
- 다음 장에는 오차역전파법으로 고속으로 기울기를 계산하는 법을 배운다.